# Hazard: Bush Fire

This class is used to model the bushfire hazard using the historical data available and creating probabilistic events for each historical event. 

The historical data used comes from the [Fire Information for Resource Management System (FIRMS)](https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/about-firms). They collect temperatures from the following satellite instruments:

    - Moderate Resolution Imaging Spectroradiometer  (MODIS): Near real time or standard quality data with 1 km resolution. Data available from November 2000 to present.

    - Visible Infrared Imaging Radiometer Suite (VIIRS): Near real time data with 0.375 km resolution. Data available from 20 January 2012 to present.

The data should be obtained at https://firms.modaps.eosdis.nasa.gov/download/ and saved as .csv file. Approximatly 15 min after submitting the request, the data can be downloaded by checking the request status. 

The `BushFire` class inherits from the `Hazard` class and has an associated hazard type `BF`. It provides a `set_hist_events()` method which enables to fill the hazard with historical events, a `set_hist_event_year_set()` method which aggregates historic events to event years, and a `set_probabilistic_event_year_set()` method which generates random event years. `hull_burned_area()` can be used to approximate the burned area of an event. It approximates a concave hull containing all the burned centroids and computes its area in hectares.

Please note: as compared to other hazards (i.e. TC) there's no clear definition on what an event is for Bush Fires. While some countries (i.e. US, Australia) provide very detailed reports, other's (i.e. Chile, Indonesia) rather provide information on the whole fire season. Thus, we produce probabilistic event years rather than single events to ensure global consistency. 

## HISTORICAL EVENTS

In June 2017, Portugal was hit by a series of deadly wild fires which led to 66 fatalities, more than 200 mn. USD damages and 45'000 ha burned area. With `set_hist_events()` we can model the fires in Portugal in 2017.

An event is defined when the temporal and spatial distance of the burning centroids is close enough. There are two attributes of the class which define the "closeness":

    - days_thres (int): temporal distance in days. Default: 2.
    - clus_thres (int): factor to multiply to the centroids resolution. Used to determine the cluster maximum distance between two centroids. Default: 15.
    
In addition to the `Hazard` attibutes, `BushFire` contains a `date_end` attribute which contains the ending date of each event.

In [1]:
# Fires in Portugal 2017
%matplotlib inline
import os

from climada.util import DATA_DIR
from climada.hazard import BushFire

# Data downloaded for MODIS
FIRMS = os.path.join(DATA_DIR, "demo/Portugal_firms_June_2017.csv")

# set up bush fire
bf_pt = BushFire()
bf_pt.set_hist_events(FIRMS)

# plot the maximum intensity of all the events at each centroid
bf_cal.plot_intensity(event=0)

In /opt/anaconda3/envs/climada_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/climada_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/climada_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/anaconda3/envs/climada_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/anaconda3/envs/c

ModuleNotFoundError: No module named 'climada'

We can focus on the region around Coimbra where the most deadly fire occured.

A given set of points or grid can be provided using the `Centroids` class. The events will be interpolated to the provided coordinates. 

In [2]:
from climada.hazard import Centroids
from climada.util.constants import ONE_LAT_KM

# focus on Coimbra region with 1.0 km resolution (MODIS resolution)
res = 1.0/ONE_LAT_KM
centr_zoom = Centroids()
centr_zoom.set_raster_from_pnt_bounds((-8.5, 39.8, -7.5, 40.2), res)

bf_zoom = BushFire()
bf_zoom.set_hist_events(FIRMS, centroids=centr_zoom)
bf_zoom.plot_intensity(0)

ModuleNotFoundError: No module named 'climada'

In [3]:
print('Burned area of Coimbra fires:', bf_zoom.hull_burned_area(1), ' ha.')

NameError: name 'bf_zoom' is not defined

See more infos on [Wikipedia](https://en.wikipedia.org/wiki/June_2017_Portugal_wildfires) around the fire.

In [6]:
## HISTORIC EVENT YEARS
To ensure global consistency, we want to generate event year sets. This is done using the method `set_hist_event_year_set()`.
These event years can later be compared to the probabilistic event years for a seamless risk analysis.
Note: all the underlying events of a year are merged into one single event which is saved.
The original events, that result from the clustering are not kept in order to save space. However, this can be done by setting keep_all_events to true

SyntaxError: invalid syntax (<ipython-input-6-84c1198eae76>, line 2)

In [7]:
# load FIRMS data for 2016-2018 an select main land Portugal only
firms = pd.read_csv(os.path.join(DATA_DIR, "demo/Portugal_firms_2016_17_18_MODIS.csv"))
firms = firms[firms['latitude']>35.]
firms = firms[firms['longitude']>-12.]


bf_years = BushFire()
bf_years.set_hist_event_year_set(firms, centr_res_factor=1/1.)

print('Events are now named according to their event year:', bf_years.event_name)
print('The number of underyling events is saved as well:', bf_years.n_events)

# plot the first event year (2016)
bf_years.plot_intensity(1)

NameError: name 'pd' is not defined

## PROBABILISTIC EVENT YEARS

`set_probabilistic_event_year_set()` generates synthetic event years.

The probabilistic even year set is generate the following way:
    1. A fire propagation probability matrix is defined
        - this propagation matrix defines the probabilitiy for fire propagation on each centroid
        - if not further specified, the observation of historic data is used - the fire can only propagate on centroids that burned in the past, including a blurr around these centroids
        - this matrix can be used to include additional information (i.e. on vegetation/land-use/elevation/population density)
    2. A random number n of events that occured in the probabilistic year is chosen.
        - If nothing else is specified, a random number within the range of n_events (the number of historic events is drawn)
        - The range can also be specified in the input
    3. n synthetic events are generated following this logic:
        - A random fire ignition point is selected (the point needs to be burnable on the fire propagation matrix)
        - The fire is then propagated from the ignition point as a cellular automat following these rules :

        1. an neighbouring centroid to the burning centroid becomes a burning centroid with a probability p. This probability is calculated as the product of a overall fire propagation probability (default=0.21) and the centroid specific factor (which is defined on the fire propagation probabilit matrix)
        2. an already burning centroid becomes an ember centroid (does not propagate fire anymore but is labled as burned)
        3. an ember centroid stays an ember centroid
        4. The propagation stops when there is no burning centroid left.
        5. The temperature provided to each centroid is randomly chosen from the corresponding historical event.
        
    4. These n synthetic events are summarized into a probabilistic event year set which is appended to the historic year sets 

Note: In the default no additional external data is included. The probabilistic event thus soley relies on past experience which likely leads to an underestimation of the current and future fire risk. The frame work allows to include climate change effects by:
    - defining the fire propagation probability matrix (using more data as described above)
    - varying the overall fire propagation probability (by i.e. linking it to a fire weather index)
    - varying the number of events per year (by i.e. linking it to a fire weather index)

In [8]:
# generate 1 probabilistic event year for Portugal
bf_years.set_probabilistic_event_year_set(n_event_years=1)
print('The probabilistic event year is appended to the historic:', bf_years.event_name)

# Plot the synthetic event year
bf_years.plot_intensity(event=4)

NameError: name 'bf_years' is not defined

In [9]:
## Some additional notes
# The fire propagation probability matrix is saved on the centroids and thus, can also be accessed on them.
# Due to the cellular automate, the matrix must be rectangular and matching the shape of the centroids from
# the historic events. 

fppm = bf_years.centroids.fire_propa_matrix
print('Shape of the matrix:' fppm.shape)
- The overall fire propagation probability can easily be modified

- The range of the number of events for a synthetic year can be modi

SyntaxError: invalid syntax (<ipython-input-9-b72e0de87dda>, line 7)

In [ ]:
# The overall fire propagation probability can easily be modified (default is 0.21)
bf_years.prop_proba = 0.18

In [ ]:
# The range of the number of events for a synthetic year can be modified when creating the event years
ign_range = [125,250]
bf_years.set_probabilistic_event_year_set(n_event_years=1, n_iginitions=ign_range)
bf_years.plot_intensity(event=5)
